<a href="https://colab.research.google.com/github/mirovoy465/colab/blob/main/ods.ai%20linear%20course%20homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Open ML Course: Линейные модели 2023**

# Домашнее задание 1

# Вопрос 1.
Создайте данные используя make_regression(n_samples=100, n_features=20, noise=10, random_state=42)
Что можно сказать о полученных данных?

In [ ]:
from sklearn.datasets import make_regression

X, y, coefs =  make_regression(n_samples=100, n_features=20, noise=10, coef = True)

Среднее признаков по модулю меньше единицы и стандартное отклонение около единицы (отличается не более, чем на 20%)

In [ ]:
X.mean(), X.std()

(0.006889835395167086, 0.995766715196734)

Есть корреляции больше .5 между разными признаками

In [ ]:
import numpy as np

correlation_matrix = np.corrcoef(X, rowvar=False)
correlation_matrix -= (np.isclose(correlation_matrix, 1)).astype(int)
(correlation_matrix > .5).any()

False

Для вычисления значений признаков требуется регуляризованная регрессия

In [ ]:
# ?????? зачем вычислять признаки

Для лучшего предсказания признаки нужно отнормировать


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
scores = np.array([])

for iter in range(1000):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3)

  scaler = StandardScaler()
  X_train_sc = scaler.fit_transform(X_train)
  X_test_sc = scaler.fit_transform(X_test)

  linreg = LinearRegression()
  fit = linreg.fit(X_train, y_train)
  fit_sc = linreg.fit(X_train_sc, y_train)

  iter_scores = r2_score(fit.predict(X_test), y_test),\
                r2_score(fit.predict(X_test_sc), y_test)

  scores = np.append(scores, iter_scores, axis = 0)

scores.reshape(-1, 2).mean(axis=0)

array([0.97623336, 0.9246088 ])

без нормализации действительно получается лучше

# Вопрос 2.

Используя sklearn.linear_model постройте LinearRegression, Ridge(random_state=1), Lasso на дефолтных параметрах и ненормированных признаках. Для оценки регрессии используем R2 из функции score модели.

Ответьте на вопросы 1-3.

Теперь отнормируйте признаки. Постройте опять 3 регрессии.

Ответьте на вопросы 4-6.

Вопрос 7 относится ко всем построенным в вопросе регрессиям.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

lr = LinearRegression()
ridge = Ridge(random_state = 1)
lasso = Lasso()

fit_lr = lr.fit(X, y)
fit_ridge = ridge.fit(X, y)
fit_lasso = lasso.fit(X, y)

print("Non-normalized: ", round(fit_lr.score(X, y), 4),
                          round(fit_ridge.score(X, y), 4),
                          round(fit_lasso.score(X, y), 4))

scaler = StandardScaler()
X_sc = scaler.fit_transform(X)

fit_lr_sc = lr.fit(X_sc, y)
fit_ridge_sc = ridge.fit(X_sc, y)
fit_lasso_sc = lasso.fit(X_sc, y)

print("Normalized: ", round(fit_lr_sc.score(X_sc, y), 4),
                      round(fit_ridge_sc.score(X_sc, y), 4),
                      round(fit_lasso_sc.score(X_sc, y), 4))

Non-normalized:  0.9979 0.9978 0.9975
Normalized:  0.9979 0.9978 0.9975


1. По score (R2) Ridge лучше всего
2. По score (R2) LinearRegression лучше всего
3. По score (R2) Lasso лучше всего
4. По score (R2) Ridge лучше всего
5. По score (R2) LinearRegression лучше всего
6. По score (R2) Lasso лучше всего
7. Из всех построенных регрессий в вопросе, Ridge лучше всего

Правильные ответы - 2 и 5

# Вопрос 3.
Из вопроса 2 используйте LinearRegression, Ridge, Lasso, построенные на признаках без нормирования. Для ответов на вопросы сравните полученные коэффициенты.

Для LinearRegression посчитайте значимость признаков, используя statsmodels. Достаньте истинные значения признаков из make_regression.

Для вопросов 5 и 6 формула расчета:
d = abs(round(reg.coef_[i],3) - round(coefs[i],3))/round(coefs[i],3)
if d == np.inf: d=1
if d == np.nan: d=0
coefs - истинные коэффициенты.

1. Значение коэффициента у константы лучше всего оценила LinearRegression (без проверки значимости)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(fit_lr.coef_, coefs),\
mean_squared_error(fit_ridge.coef_, coefs),\
mean_squared_error(fit_lasso.coef_, coefs)

(10.91961025861889, 11.940369659275188, 12.463900619701826)

2. Lasso правильно занулила все нулевые признаки

In [ ]:
fit_lasso.coef_.round(), coefs.round()

(array([82.,  0.,  0., 42., 26., -1., -0., -0., 73.,  3.,  0., 65., 16.,
         0.,  0., 61.,  0., 88., 29., 83.]),
 array([74.,  0.,  0., 46., 28.,  0.,  0.,  0., 77.,  0.,  0., 72., 21.,
         0.,  0., 63.,  0., 90., 32., 89.]))

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y, X).fit()
print(model.summary()), coefs.round(4).reshape(-1,1)


                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                              1906.
Date:                Wed, 04 Oct 2023   Prob (F-statistic):                    1.40e-98
Time:                        06:27:58   Log-Likelihood:                         -362.22
No. Observations:                 100   AIC:                                      764.4
Df Residuals:                      80   BIC:                                      816.5
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

(None,
 array([[73.9104],
        [ 0.    ],
        [ 0.    ],
        [46.4767],
        [28.4239],
        [ 0.    ],
        [ 0.    ],
        [ 0.    ],
        [76.8691],
        [ 0.    ],
        [ 0.    ],
        [71.9202],
        [20.6836],
        [ 0.    ],
        [ 0.    ],
        [63.1335],
        [ 0.    ],
        [90.1118],
        [31.641 ],
        [89.4811]]))

4. У Ridge и Lasso коэффициенты всегда меньше, чем у LinearRegression

In [ ]:
(fit_ridge.coef_ < fit_lr.coef_).all(), (fit_lasso.coef_ < fit_lr.coef_).all()

(False, False)

5. LinearRegression точнее всего определила коэффициенты, если посчитать сумму долей абсолютных отклонений от истинного значения

In [ ]:
sum = 0
sum_ridge = 0
sum_lasso = 0
for i in range(len(coefs)):

  if coefs[i] == 0:
    sum+=1
    sum_ridge+=1
    sum_lasso+=1
    continue

  if fit_lr.coef_[i] != 0: sum += abs((fit_lr.coef_[i] - coefs[i])/coefs[i])
  if fit_ridge.coef_[i] != 0: sum_ridge += abs((fit_ridge.coef_[i] - coefs[i])/coefs[i])
  if fit_lasso.coef_[i] != 0: sum_lasso += abs((fit_lasso.coef_[i] - coefs[i])/coefs[i])

sum, sum_ridge, sum_lasso

(10.681956274276697, 10.742182245978485, 10.892414814801977)

#Вопрос 4.
Давайте теперь разберем, как правильно нормировать Х. Изменим немного данные:

X['x5'] = X['x5'] + 5

X['x10'] = X['x10'] + 10

X['x15'] = X['x15'] + 15

(Порядковый номер столбца Х с 0)

Разобьем выборку на train - где мы будем обучать регрессию и test - где мы будем проверять качество регрессии:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

В качестве реализации регрессии возьмем Ridge(random_state=1). Метрика качества R2.

In [ ]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

X, y, coefs =  make_regression(n_samples=100, n_features=20, noise=10, coef = True)

X.T[5] += 5
X.T[10] += 10
X.T[15] += 15

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

scaler = StandardScaler()
X_train_sc_train =  scaler.fit(X_train).transform(X_train)
X_test_sc_train =   scaler.fit(X_train).transform(X_test)
X_test_sc_test =    scaler.fit(X_test).transform(X_test)
X_train_sc_X =      scaler.fit(X).transform(X_train)
X_test_sc_X =       scaler.fit(X).transform(X_test)


fit_ridge = Ridge(random_state=1).fit(X_train, y_train)
fit_ridge_sc_train = Ridge(random_state=1).fit(X_train_sc_train, y_train)
fit_ridge_sc_X = Ridge(random_state=1).fit(X_train_sc_X, y_train)

round(r2_score(fit_ridge.predict(X_test), y_test),4),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_train), y_test),4),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_test), y_test),4),\
round(r2_score(fit_ridge_sc_X.predict(X_test_sc_X), y_test),4)

(0.9925, 0.9925, 0.8917, 0.9925)

#Вопрос 5.
Давайте теперь изменим только X_test (моделируя ситуацию, когда тестовые данные "поплыли"):

X_test['x5'] = X_test['x5'] - 2

X_test['x10'] = X_test['x10'] - 5

X_test['x15'] = X_test['x15'] - 7

В качестве реализации регрессии продолжаем с Ridge(random_state=1). Метрика качества R2.

In [ ]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

X, y, coefs =  make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

X_test.T[5] -= 2
X_test.T[10] -= 5
X_test.T[15] -= 7

scaler = StandardScaler()
X_train_sc_train =  scaler.fit(X_train).transform(X_train)
X_test_sc_train  =  scaler.fit(X_train).transform(X_test)
X_test_sc_test   =  scaler.fit(X_test).transform(X_test)
X_train_sc_X     =  scaler.fit(X).transform(X_train)
X_test_sc_X      =  scaler.fit(X).transform(X_test)

fit_ridge          =  Ridge(random_state=1).fit(X_train, y_train)
fit_ridge_sc_train =  Ridge(random_state=1).fit(X_train_sc_train, y_train)
fit_ridge_sc_X     =  Ridge(random_state=1).fit(X_train_sc_X, y_train)

round(r2_score(fit_ridge.predict(X_test), y_test), 2),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_train), y_test), 2),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_test), y_test), 2),\
round(r2_score(fit_ridge_sc_X.predict(X_test_sc_X), y_test), 2)

(-0.75, -0.74, 0.94, -0.75)

#Вопрос 6.
Давайте теперь изменим X_test и X_train следующим образом:

X_train['x5'] = X_train['x5'] * 2.5

X_train['x10'] = X_train['x10'] * 5.5

X_train['x15'] = X_train['x15'] * 7.7
_____________________________________

X_test['x5'] = X_test['x5'] * 2 + 0.5

X_test['x10'] = X_test['x10'] * 5 + 1

X_test['x15'] = X_test['x15'] * 7 + 1.5

В качестве реализации регрессии продолжаем с Ridge(random_state=1). Метрика качества R2.

In [ ]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

X, y, coefs =  make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

X_train.T[5]  = X_train.T[5] * 2.5
X_train.T[10] = X_train.T[10] * 5.5
X_train.T[15] = X_train.T[15] * 7.7
X_test.T[5]   = X_test.T[5] * 2 + 0.5
X_test.T[10]  = X_test.T[10] * 5 + 1
X_test.T[15]  = X_test.T[15] * 7 + 1.5

scaler = StandardScaler()
X_train_sc_train =  scaler.fit(X_train).transform(X_train)
X_test_sc_train  =  scaler.fit(X_train).transform(X_test)
X_test_sc_test   =  scaler.fit(X_test).transform(X_test)
X_train_sc_X     =  scaler.fit(X).transform(X_train)
X_test_sc_X      =  scaler.fit(X).transform(X_test)

fit_ridge          =  Ridge(random_state=1).fit(X_train, y_train)
fit_ridge_sc_train =  Ridge(random_state=1).fit(X_train_sc_train, y_train)
fit_ridge_sc_X     =  Ridge(random_state=1).fit(X_train_sc_X, y_train)

round(r2_score(fit_ridge.predict(X_test), y_test), 2),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_train), y_test), 2),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_test), y_test), 2),\
round(r2_score(fit_ridge_sc_X.predict(X_test_sc_X), y_test), 2)

(0.99, 0.99, 0.94, 0.98)

#Вопрос 7.
Работаем с датасетом, определенном в вопросе 4 (где 5й,10й,15й признаки увеличены). Разобьём на трейн и тест как в вопросе 4. Используем нормированные признаки и Ridge(random_state=1):

Теперь давайте сделаем пропущенные значения для важного признака:

X_train.loc[X_train['x1'].isin(X_train['x1'].sample(smpl, random_state=1+exp)), 'x1'] = np.nan,
где попробуем 5 вариантов случайности exp = range(5). smpl - кол-во пропущенных значений.

Ответьте на вопросы 1-2.

Потом сделаем пропущенные значения только для неважного признака:
X_train.loc[X_train['x10'].isin(X_train['x10'].sample(smpl, random_state=1+exp)), 'x10'] = np.nan,
где попробуем 5 вариантов случайности exp = range(5).

Ответьте на вопросы 3-4.

In [ ]:
'''ТЕСТИРОВАНИЕ ДЛЯ ВАЖНОГО ПРИЗНАКА'''

from sklearn.linear_model import Ridge
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

X, y, coefs =  make_regression(n_samples=100, n_features=20, noise=10, coef = True)

X.T[5] += 5
X.T[10] += 10
X.T[15] += 15

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

columns = [f'x{i}' for i in range(20)]
X_train = pd.DataFrame(X_train, columns = columns)

smpl = 60
train = pd.concat([X_train,pd.DataFrame(y_train,columns=['y'])],axis=1)
train.loc[train['x1'].isin(train['x1'].sample(smpl, random_state=2)), 'x1'] = np.nan
# train.dropna(inplace=True)
train.fillna(np.mean(train.drop('y',axis=1)),inplace=True)
y_train = train['y']
X_train = train.drop('y',axis=1)

scaler = StandardScaler()
X_train_sc_train =  scaler.fit(X_train).transform(X_train)
X_test_sc_train =   scaler.fit(X_train).transform(X_test)
X_test_sc_test =    scaler.fit(X_test).transform(X_test)
X_train_sc_X =      scaler.fit(X).transform(X_train)
X_test_sc_X =       scaler.fit(X).transform(X_test)


fit_ridge = Ridge(random_state=1).fit(X_train, y_train)
fit_ridge_sc_train = Ridge(random_state=1).fit(X_train_sc_train, y_train)
fit_ridge_sc_X = Ridge(random_state=1).fit(X_train_sc_X, y_train)

round(r2_score(fit_ridge.predict(X_test), y_test),4),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_train), y_test),4),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_test), y_test),4),\
round(r2_score(fit_ridge_sc_X.predict(X_test_sc_X), y_test),4)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


(0.9966, 0.9967, 0.9451, 0.9966)

In [ ]:
'''ТЕСТИРОВАНИЕ ДЛЯ МЕНЕЕ ВАЖНОГО ПРИЗНАКА'''

from sklearn.linear_model import Ridge
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

X, y, coefs =  make_regression(n_samples=100, n_features=20, noise=10, coef = True)

X.T[5] += 5
X.T[10] += 10
X.T[15] += 15

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

columns = [f'x{i}' for i in range(20)]
X_train = pd.DataFrame(X_train, columns = columns)

smpl = 60
train = pd.concat([X_train,pd.DataFrame(y_train,columns=['y'])],axis=1)
train.loc[train['x10'].isin(train['x10'].sample(smpl, random_state=2)), 'x10'] = np.nan

train.dropna(inplace=True)
# train.fillna(np.mean(train.drop('y',axis=1)),inplace=True)

y_train = train['y']
X_train = train.drop('y',axis=1)

scaler = StandardScaler()
X_train_sc_train =  scaler.fit(X_train).transform(X_train)
X_test_sc_train =   scaler.fit(X_train).transform(X_test)
X_test_sc_test =    scaler.fit(X_test).transform(X_test)
X_train_sc_X =      scaler.fit(X).transform(X_train)
X_test_sc_X =       scaler.fit(X).transform(X_test)


fit_ridge = Ridge(random_state=1).fit(X_train, y_train)
fit_ridge_sc_train = Ridge(random_state=1).fit(X_train_sc_train, y_train)
fit_ridge_sc_X = Ridge(random_state=1).fit(X_train_sc_X, y_train)

round(r2_score(fit_ridge.predict(X_test), y_test),4),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_train), y_test),4),\
round(r2_score(fit_ridge_sc_train.predict(X_test_sc_test), y_test),4),\
round(r2_score(fit_ridge_sc_X.predict(X_test_sc_X), y_test),4)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


(-1.3594, -1.3006, -0.9206, -1.4797)

#Вопрос 9.
Создадим данные, используя
X, y, coefs = make_regression(n_samples=1000, n_features=200, noise=10, random_state=42, coef=True)

Давайте построим Ridge(random_state=1), Lasso, LinearRegression на ненормированных данных (т.к. сырые данные близки к нормированным) и на 100 случайно выбранных наблюдениях. Качество оценим по mean_squared_error от истинных значений и предсказаний на наблюдениях, которые не участвовали в обучении (900).

In [4]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


X, y, coefs = make_regression(n_samples=1000, n_features=200, noise=10, random_state=42, coef=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.9, random_state=42)

r_fit = Ridge(random_state=1).fit(X_train,y_train)
l_fit = Lasso().fit(X_train,y_train)
lr_fit = LinearRegression().fit(X_train,y_train)

print(
      mean_squared_error(r_fit.predict(X_test),y_test),\
      mean_squared_error(l_fit.predict(X_test),y_test),\
      mean_squared_error(lr_fit.predict(X_test),y_test))

print(
      mean_absolute_error(coefs,r_fit.coef_),
      mean_absolute_error(coefs,l_fit.coef_),
      mean_absolute_error(coefs,lr_fit.coef_))


20990.913124897692 140.09059281921682 21049.257544778877
6.649737949291521 0.17095310276609033 6.692372273451126


#Вопрос 10.
Мультиколлинеарность. Вернемся к первоначальным данным:
X, y, coefs = make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef=True)

Добавим к X столбец x['x20'] = x['x1'] (столбец, равный первому признаку, если нумерация столбцов с 0).
Давайте построим Ridge(random_state=1), Lasso, LinearRegression на нормированных данных.

Ответьте на вопросы 1-3.

Пропуск важной переменной. Вернемся к первоначальным данным:
X, y, coefs = make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef=True)
Уберем из X важный x['x1'] (столбец, равный первому признаку, если нумерация признаков с 0).
Давайте построим Ridge(random_state=1), Lasso, LinearRegression на нормированных данных.

Ответьте на вопросы 4-6.

In [38]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X, y, coefs = make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef=True)

columns = [f'x{i}' for i in range(20)]
X_mult = pd.DataFrame(X.copy(), columns = columns)
X_mult['x20'] = X_mult['x1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.9, random_state=42)
X_mult_train, X_mult_test, _, _ = train_test_split(X_mult, y, test_size=.9, random_state=42)

scaler = StandardScaler()
X_train_sc =  scaler.fit(X_train).transform(X_train)
X_test_sc =   scaler.fit(X_train).transform(X_test)
X_mult_train_sc =  scaler.fit(X_mult_train).transform(X_mult_train)
X_mult_test_sc =   scaler.fit(X_mult_train).transform(X_mult_test)

r_fit = Ridge(random_state=1).fit(X_train_sc,y_train)
l_fit = Lasso().fit(X_train_sc,y_train)
lr_fit = LinearRegression().fit(X_train_sc,y_train)
r_mult_fit = Ridge(random_state=1).fit(X_mult_train_sc,y_train)
l_mult_fit = Lasso().fit(X_mult_train_sc,y_train)
lr_mult_fit = LinearRegression().fit(X_mult_train_sc,y_train)

print(
      coefs.round(2).reshape(-1,5),'\nridge\n',\
      r_fit.coef_.round(2).reshape(-1,5), '\n',r_mult_fit.coef_.round(2).reshape(-1,7),'\nlasso\n',\
      l_fit.coef_.round(2).reshape(-1,5), '\n',l_mult_fit.coef_.round(2).reshape(-1,7),'\nlinear\n',\
      lr_fit.coef_.round(2).reshape(-1,5), '\n',lr_mult_fit.coef_.round(2).reshape(-1,7)
      )


[[ 6.6  94.77 40.71  0.    0.  ]
 [ 0.   11.1  95.51 80.82 34.88]
 [ 0.    0.    0.    0.    0.  ]
 [29.96  7.23  0.    0.   52.26]] 
ridge
 [[ 20.4   21.17  -2.09 -10.66  12.42]
 [  1.79   0.65  13.03   4.91   5.41]
 [  2.4    7.58   3.9    5.63  12.19]
 [ 22.13 -24.5    5.7    1.26   2.21]] 
 [[ 21.65  14.01  -2.61  -9.61   9.85   1.73   1.65]
 [ 14.35   5.48   4.11   1.7    7.88   3.29   2.81]
 [ 12.19  21.58 -21.26   3.72   2.04   4.25  14.01]] 
lasso
 [[  1.72  23.13   0.    -4.53   0.  ]
 [  0.    -0.    24.89   3.31  -0.  ]
 [  0.     4.52  -0.     0.    17.37]
 [ 30.63 -41.09  -0.     0.    -0.  ]] 
 [[  1.72  23.13   0.    -4.53   0.     0.    -0.  ]
 [ 24.89   3.31  -0.     0.     4.52  -0.     0.  ]
 [ 17.37  30.63 -41.09  -0.     0.    -0.     0.  ]] 
linear
 [[ 22.17  23.36  -2.08 -11.53  14.14]
 [  1.45   1.    14.61   7.4    3.65]
 [  1.67   8.41   4.03   6.17  12.74]
 [ 23.46 -26.74   7.04   0.3    1.23]] 
 [[ 23.5   15.1   -2.71 -10.32  11.05   1.42   2.04]
 [ 16.02   

Пропуск важной переменной. Вернемся к первоначальным данным:
X, y, coefs = make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef=True)
Уберем из X важный x['x1'] (столбец, равный первому признаку, если нумерация признаков с 0).
Давайте построим Ridge(random_state=1), Lasso, LinearRegression на нормированных данных.

In [44]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score


X, y, coefs = make_regression(n_samples=100, n_features=20, noise=10, random_state=42, coef=True)
X_del = np.delete(X,1,axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.9, random_state=42)
X_del_train, X_del_test, _, _ = train_test_split(X_del, y, test_size=.9, random_state=42)

scaler = StandardScaler()
X_train_sc =  scaler.fit(X_train).transform(X_train)
X_test_sc =   scaler.fit(X_train).transform(X_test)
X_del_train_sc =  scaler.fit(X_del_train).transform(X_del_train)
X_del_test_sc =   scaler.fit(X_del_train).transform(X_del_test)

r_fit = Ridge(random_state=1).fit(X_train_sc,y_train)
l_fit = Lasso().fit(X_train_sc,y_train)
lr_fit = LinearRegression().fit(X_train_sc,y_train)
r_del_fit = Ridge(random_state=1).fit(X_del_train_sc,y_train)
l_del_fit = Lasso().fit(X_del_train_sc,y_train)
lr_del_fit = LinearRegression().fit(X_del_train_sc,y_train)

print(
      coefs.round(2).reshape(-1,5),'\nridge\n',\
      r_fit.coef_.round(2).reshape(-1,5), '\n',r_del_fit.coef_.round(2),'\nlasso\n',\
      l_fit.coef_.round(2).reshape(-1,5), '\n',l_del_fit.coef_.round(2),'\nlinear\n',\
      lr_fit.coef_.round(2).reshape(-1,5), '\n',lr_del_fit.coef_.round(2)
      )
print(
      r2_score(r_fit.predict(X_test_sc),y_test)/          \
      r2_score(r_del_fit.predict(X_del_test_sc),y_test),  \
      r2_score(l_fit.predict(X_test_sc),y_test)/          \
      r2_score(l_del_fit.predict(X_del_test_sc),y_test),  \
      r2_score(lr_fit.predict(X_test_sc),y_test)/         \
      r2_score(lr_del_fit.predict(X_del_test_sc),y_test))


[[ 6.6  94.77 40.71  0.    0.  ]
 [ 0.   11.1  95.51 80.82 34.88]
 [ 0.    0.    0.    0.    0.  ]
 [29.96  7.23  0.    0.   52.26]] 
ridge
 [[ 20.4   21.17  -2.09 -10.66  12.42]
 [  1.79   0.65  13.03   4.91   5.41]
 [  2.4    7.58   3.9    5.63  12.19]
 [ 22.13 -24.5    5.7    1.26   2.21]] 
 [[ 16.51  -0.45 -13.92  20.39   1.99  -2.44   8.93   3.14   9.44   4.57
    6.65   5.81  14.37  12.22  23.83 -34.52  11.83  -1.16  -4.08]] 
lasso
 [[  1.72  23.13   0.    -4.53   0.  ]
 [  0.    -0.    24.89   3.31  -0.  ]
 [  0.     4.52  -0.     0.    17.37]
 [ 30.63 -41.09  -0.     0.    -0.  ]] 
 [[  0.     0.   -29.77  22.91   0.    -3.63   7.38  -0.     0.     0.
    7.67   0.     0.     0.    38.49 -48.43   7.88  -0.    -0.  ]] 
linear
 [[ 22.17  23.36  -2.08 -11.53  14.14]
 [  1.45   1.    14.61   7.4    3.65]
 [  1.67   8.41   4.03   6.17  12.74]
 [ 23.46 -26.74   7.04   0.3    1.23]] 
 [[ 17.64   0.07 -15.65  24.67   1.55  -2.55   9.8    5.88   8.02   4.04
    7.42   6.6   17.06  12.93